In [397]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

In [398]:
booking_df = pd.read_csv("./bookings.csv", sep=";")
booking_df.describe()

,Is Canceled,Lead Time,Arrival Date Year,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,stays total nights,Adults,Children,Babies
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119386.000000,119390.000000
mean,0.370416,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,3.427900,1.856403,0.103890,0.007949
std,0.482918,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,2.557439,0.579261,0.398561,0.097436
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,2.000000,0.000000,0.000000
50%,0.000000,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,3.000000,2.000000,0.000000,0.000000
75%,1.000000,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,4.000000,2.000000,0.000000,0.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,69.000000,55.000000,10.000000,10.000000


In [399]:
# Приведите названия колонок к нижнему регистру и замените пробелы на знак нижнего подчеркивания.
new_names = booking_df.columns.str.lower().str.replace(' ', '_')
booking_df.columns = new_names

In [400]:
booking_df['children'] = booking_df['children'].astype('Int64')

In [401]:
booking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   hotel                      119390 non-null  object
 1   is_canceled                119390 non-null  int64 
 2   lead_time                  119390 non-null  int64 
 3   arrival_full_date          119390 non-null  object
 4   arrival_date_year          119390 non-null  int64 
 5   arrival_date_month         119390 non-null  object
 6   arrival_date_week_number   119390 non-null  int64 
 7   arrival_date_day_of_month  119390 non-null  int64 
 8   stays_in_weekend_nights    119390 non-null  int64 
 9   stays_in_week_nights       119390 non-null  int64 
 10  stays_total_nights         119390 non-null  int64 
 11  adults                     119390 non-null  int64 
 12  children                   119386 non-null  Int64 
 13  babies                     119390 non-null  

In [402]:
# Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.
booking_df[booking_df['is_canceled'] == 0].groupby('country', as_index=False).aggregate({'is_canceled': 'count'}).sort_values(by='is_canceled', ascending=False).head(5)

,country,is_canceled
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069


In [403]:
# На сколько ночей в среднем бронируют отели разных типов?
booking_df.groupby('hotel', as_index=False).aggregate({'stays_total_nights': 'mean'})


,hotel,stays_total_nights
0,City Hotel,2.978142
1,Resort Hotel,4.318547


In [404]:
# Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга. Сколько подобных наблюдений встретилось в датасете?
booking_df[booking_df['reserved_room_type'] != booking_df['assigned_room_type']].shape[0]

14917

In [405]:
# Проанализируйте даты запланированного прибытия. 
# На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?
popular_month_2016 = booking_df[(booking_df['arrival_date_year'] == 2016) & (booking_df['is_canceled'] == 0)].groupby('arrival_date_month', as_index=False).aggregate({'is_canceled': 'count'}).sort_values(by=['is_canceled'], ascending=False).head(1)
popular_month_2017 = booking_df[(booking_df['arrival_date_year'] == 2017) & (booking_df['is_canceled'] == 0)].groupby('arrival_date_month', as_index=False).aggregate({'is_canceled': 'count'}).sort_values(by=['is_canceled'], ascending=False).head(1)

f'2016: month - {popular_month_2016["arrival_date_month"].values[0]}, number of books: {popular_month_2016["is_canceled"].values[0]}; 2017: month - {popular_month_2017["arrival_date_month"].values[0]}, number of books: {popular_month_2017["is_canceled"].values[0]}'

'2016: month - October, number of books: 3689; 2017: month - May, number of books: 3551'

In [406]:
# Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов.
canceled_year = booking_df[(booking_df['hotel'] == 'City Hotel') & (booking_df['is_canceled'] == 1)].groupby(['arrival_date_year', 'arrival_date_month'], as_index=False).aggregate({'is_canceled' : 'sum'})
canceled_year_2015 = canceled_year[canceled_year['arrival_date_year'] == 2015][['is_canceled', 'arrival_date_month']].sort_values(by=['is_canceled'], ascending=False).head(1)
canceled_year_2016 = canceled_year[canceled_year['arrival_date_year'] == 2016][['is_canceled', 'arrival_date_month']].sort_values(by=['is_canceled'], ascending=False).head(1)
canceled_year_2017 = canceled_year[canceled_year['arrival_date_year'] == 2017][['is_canceled', 'arrival_date_month']].sort_values(by=['is_canceled'], ascending=False).head(1)
f"Number of cancels: 2015 - month: {canceled_year_2015.iloc[0]['arrival_date_month']}, {canceled_year_2015.iloc[0]['is_canceled']}; 2016 - month: {canceled_year_2016.iloc[0]['arrival_date_month']}, {canceled_year_2016.iloc[0]['is_canceled']}; 2017 - month: {canceled_year_2017.iloc[0]['arrival_date_month']}, {canceled_year_2017.iloc[0]['is_canceled']}"


'Number of cancels: 2015 - month: September, 1543; 2016 - month: October, 1947; 2017 - month: May, 2217'

In [407]:
# Посмотрите на числовые характеристики трёх переменных: adults, children и babies. Какая из них имеет наибольшее среднее значение?
booking_df[['adults', 'children', 'babies']].describe().loc['mean']

adults      1.856403
children     0.10389
babies      0.007949
Name: mean, dtype: Float64

In [408]:
# Создайте колонку total_kids, объединив children и babies. Отели какого типа в среднем пользуются большей популярностью у клиентов с детьми?
booking_df['total_kids'] = booking_df['children']+booking_df['babies']
booking_df.groupby('hotel', as_index=False).agg({'total_kids': 'mean'})
booking_df['total_kids'].dropna(inplace=True)
booking_df['total_kids'].fillna(0, inplace=True)





In [409]:
# Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), и False – в противном случае.

def kids(x) -> bool:
    if x > 0:
        return True
    else: 
        return False

booking_df['has_kids'] = booking_df['total_kids'].apply(kids)





In [410]:
booking_df[['is_canceled', 'has_kids']].value_counts()

is_canceled  has_kids
0            False       69093
1            False       40965
0            True         6073
1            True         3259
Name: count, dtype: int64

In [411]:
# Посчитайте отношение количества ушедших пользователей к общему количеству клиентов, выраженное в процентах (churn rate). Укажите, среди какой группы показатель выше.
(booking_df[['is_canceled', 'has_kids']].value_counts()/booking_df['has_kids'].value_counts())*100



is_canceled  has_kids
0            False       62.778717
1            False       37.221283
0            True        65.077154
1            True        34.922846
Name: count, dtype: float64